# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'https?://[^\s]+' , ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[a-zA-Z0-9]*' , ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@[a-zA-Z0-9]*' , ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z]' , ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens if len(w) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

tokenized = [preprocess(twit) for twit in messages]
idx = [idx for idx, tokens in enumerate(tokenized) if len(tokens) >0]
tokenized = [tokenized[i] for i in idx]
sentiments =  [sentiments[i] for i in idx]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
temp = Counter()
[temp.update(tokens) for tokens in tokenized]
bow = temp
print(bow.most_common(3))

[('the', 398911), ('to', 379518), ('is', 284906)]


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 


# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = sum(bow.values())
freqs = {word: bow[word]/total_count for word in bow}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 1e-5

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)] 


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you', 'that', 'be', 'short', 'what', 'buy']


4507

In [9]:
print(filtered_words[:10])

['great', 'ill', 'wait', 'staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting']


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [10]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
#tokenized = tokenizes
#filtered_sorted_count = [word for word in sorted(bow, key=bow.get, reverse=True) if word in filtered_words]
#filtered_sorted_count = [word for word in enumerate(filtered_words,1)]

filtered_sorted_count = [word for word in sorted(bow, key=bow.get, reverse=True) if word in filtered_words]
#filtered_sorted_count = [word for word in enumerate(filtered_words,1)]

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: idx for idx, word in enumerate(filtered_sorted_count,1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {idx :word for idx,word in enumerate(filtered_sorted_count,1)}

# tokenized with the words not in `filtered_words` removed.
#filtered = [[token for token in tokens if token in filtered_words] for tokens in tokenized]
filtered = []
for tokens in tokenized:
    filtered.append([token for token in tokens if token in filtered_words])

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [11]:
#vocab = {word: idx for idx, word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
#id2vocab = {idx :word for idx,word in enumerate(filtered_words)}

balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral  # so on an average you have (N_examples - n_neutral)/4 (other 4 groups total no /4). So you need to keep these many for the newutrral group as well 
#print((N_examples - n_neutral)/n_neutral)
#print("" + str(N_examples) + " " +  str(n_neutral))
for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [12]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19852025428639933

Finally let's convert our tokens into integer ids which we can pass to the network.

In [13]:
#print(filtered_sorted_count[1:10])

token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']





In [14]:
print(filtered_words[1:10])

print(balanced['messages'][:2])
print(vocab['jefferies'])


['ill', 'wait', 'staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting']
[['great', 'ill', 'wait'], ['staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting', 'target', 'price', 'usd', 'our', 'own', 'verdict']]
1408


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [15]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size)
        
        # Setup additional layers
        self.lstm  = nn.LSTM(input_size=embed_size, hidden_size=lstm_size, num_layers = lstm_layers, batch_first= False,
                            dropout = dropout)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(in_features=lstm_size, out_features=output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        if False:#torch.cuda.is_available():
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
            weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
             hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
             weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        #print(hidden)
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        batch_size = nn_input.size(0)
        #print(nn_input)
        embed = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embed, hidden_state)
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1]
        out = self.fc(lstm_out)
        out = self.dropout(out)
        
        soft_max_out = self.softmax(out)
        #soft_max_out = soft_max_out.view(batch_size,-1)
        #soft_max_out = soft_max_out[:, -1]
        
        return soft_max_out, hidden_state

### View Model

In [16]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
#print(model)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
#train_on_gpu=torch.cuda.is_available()

#if(train_on_gpu):
#    input= input.cuda()
#    model.cuda()

hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.9348, -1.6140, -1.2923, -1.4381, -1.9348],
        [-1.5819, -1.5454, -1.3291, -1.4505, -2.5006],
        [-1.6842, -1.3563, -1.6842, -1.6842, -1.6842],
        [-1.4249, -1.4598, -1.7911, -1.3029, -2.4230]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [17]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [18]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
cut_off1 = int(0.8 * len(token_ids))
cut_off2 = int(0.8 * len(token_ids))
print(cut_off1)
print(token_ids[1:100])
train_features = token_ids[:cut_off1]
valid_features = token_ids[cut_off2:]
train_labels = sentiments[:cut_off1]
valid_labels = sentiments[cut_off2:]
print(valid_features[:2])

824141
[[534, 1408, 1565, 6, 174, 108, 555, 155, 52, 689, 205, 269, 574], [1181, 81, 190, 135, 122, 434, 135, 91, 1371, 122, 378, 1138], [110, 137, 1449, 1227, 353, 15, 361, 214], [60, 23, 308, 313, 17, 597], [488, 41, 10, 194, 7, 820, 52, 63, 591, 68], [186, 168, 10, 111, 34, 44, 136], [137, 156, 42, 58, 847, 13, 46, 125, 152, 44, 13, 2020, 116, 4120, 2631], [148], [311, 482, 98], [52, 3617, 918, 709, 103, 119, 835, 390, 3274, 292], [272, 35, 319, 235, 148, 43, 1710, 277, 109, 457], [345, 1058, 368, 781], [78, 23, 308, 313, 17, 597], [1242, 2820, 35, 1029, 3959, 4013, 15, 233, 981, 9, 24, 542, 150, 164], [534, 4372, 1331, 388, 52, 155, 6, 174, 555, 155, 52, 689, 205, 269, 574], [91, 129, 476, 484, 32, 514, 319, 171, 6, 368, 125, 76, 81, 56, 37, 255], [39, 368, 311, 37, 2127, 987, 39, 46, 727, 1469, 1892], [87, 2, 177, 291, 2208, 20], [4], [73, 204, 33, 42, 257, 1366], [368, 311, 154, 189, 72, 307, 260], [339, 183, 9], [13, 150, 2556, 33, 42, 38, 102, 1645, 28, 52], [60, 23, 308, 313, 

In [19]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)

#if(train_on_gpu):
#    text_batch= text_batch.cuda()
#    model.cuda()

logps, hidden = model.forward(text_batch, hidden)
print(logps)

tensor([[-1.5682, -1.5798, -1.7223, -1.4308, -1.7850],
        [-1.7760, -1.6326, -1.5937, -1.5958, -1.4725],
        [-1.6653, -1.8421, -1.4886, -1.7343, -1.3857],
        [-1.7661, -1.4544, -1.7222, -1.4790, -1.6665],
        [-1.6200, -1.7956, -1.5939, -1.5652, -1.4965],
        [-1.6635, -1.4103, -1.6544, -1.6232, -1.7260],
        [-1.5292, -1.5949, -1.5949, -1.7878, -1.5602],
        [-1.9498, -1.5570, -1.5570, -1.5570, -1.4899],
        [-1.8756, -1.6175, -1.6829, -1.8061, -1.2100],
        [-1.8311, -1.4494, -1.6905, -1.4998, -1.6224],
        [-1.6364, -1.6364, -1.6364, -1.8037, -1.3813],
        [-1.8962, -1.3578, -1.9626, -1.6201, -1.3695],
        [-1.7905, -1.6039, -1.5304, -1.7565, -1.4151],
        [-1.6244, -1.7921, -1.5723, -1.5826, -1.4991],
        [-1.5682, -1.5682, -1.5682, -1.6726, -1.6767],
        [-1.7317, -1.5880, -1.5499, -1.6391, -1.5499],
        [-1.7799, -1.5567, -1.5567, -1.6151, -1.5567],
        [-1.7942, -1.5307, -1.4598, -1.7598, -1.5466],
        [-

### Training
It's time to train the neural network!

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#train_on_gpu = False
model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(4508, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [21]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 512
learning_rate = 1e-3

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()
sequence_length = 40
for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    hidden = model.init_hidden(batch_size)
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=True):
        #print(text_batch.size())
        #print(labels.size())

        if text_batch.size() != torch.Size([sequence_length, batch_size]):
            continue
        steps += 1
        #print(steps)

        hidden = tuple([h.data for h in hidden])
        for each in hidden:
            each.to(device)
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)

        # TODO Implement: Train Model
        model.zero_grad()
        logps, hidden = model.forward(text_batch, hidden)
        loss = criterion(logps.squeeze(), labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        if steps % print_every == 0:
            #print("Evaluating")
            model.eval()
            validation_losses = []
            validation_accuracy = []
            val_h = model.init_hidden(batch_size)
            # TODO Implement: Print metrics
            for val_text_batch, val_labels in dataloader(valid_features, valid_labels, batch_size=batch_size, 
                                                 sequence_length=sequence_length, shuffle=True):
                if val_text_batch.size() != torch.Size([sequence_length, batch_size]):
                    continue
                val_h = tuple([h.data for h in val_h])
                # Set Device
                for each in val_h:
                    each.to(device)

                val_text_batch, val_labels = val_text_batch.to(device), val_labels.to(device)

                val_logps, val_h = model.forward(val_text_batch, val_h)
                val_loss = criterion(val_logps.squeeze(), val_labels)
                validation_losses.append(val_loss.item())
                
                validation_ps = torch.exp(val_logps)
                top_p, top_class = validation_ps.topk(1, dim=1)
                equals = top_class == val_labels.view(*top_class.shape)
                validation_accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
                #print(validation_accuracy)
                
            model.train()
            curr_val_set_accuracy = sum(validation_accuracy)/len(validation_accuracy)
            print("Epoch: {}/{}".format(epoch+1, epochs),
                  " Step: {}".format(steps),
                  "Loss: {:.4f}".format(loss.item()),"Validation loss: {:.4f}".format(sum(validation_losses)/len(validation_losses)),
                  "Accuracy : {:.4f}".format(curr_val_set_accuracy)
                 )
            

Starting epoch 1
Epoch: 1/3  Step: 100 Loss: 1.1562 Validation loss: 1.1237 Accuracy : 0.5412
Epoch: 1/3  Step: 200 Loss: 1.0650 Validation loss: 1.0289 Accuracy : 0.5862
Epoch: 1/3  Step: 300 Loss: 1.0836 Validation loss: 0.9936 Accuracy : 0.6094
Epoch: 1/3  Step: 400 Loss: 1.0232 Validation loss: 0.9591 Accuracy : 0.6268
Epoch: 1/3  Step: 500 Loss: 1.0906 Validation loss: 0.9773 Accuracy : 0.6228
Epoch: 1/3  Step: 600 Loss: 1.0316 Validation loss: 0.9322 Accuracy : 0.6347
Epoch: 1/3  Step: 700 Loss: 1.0703 Validation loss: 0.9113 Accuracy : 0.6500
Epoch: 1/3  Step: 800 Loss: 1.0726 Validation loss: 0.9072 Accuracy : 0.6473
Epoch: 1/3  Step: 900 Loss: 1.0310 Validation loss: 0.8974 Accuracy : 0.6580
Epoch: 1/3  Step: 1000 Loss: 0.9835 Validation loss: 0.8940 Accuracy : 0.6560
Epoch: 1/3  Step: 1100 Loss: 0.9920 Validation loss: 0.8909 Accuracy : 0.6574
Epoch: 1/3  Step: 1200 Loss: 0.9624 Validation loss: 0.8813 Accuracy : 0.6593
Epoch: 1/3  Step: 1300 Loss: 0.9848 Validation loss: 0.8

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [22]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)
    # Get the NN output
    hidden = model.init_hidden(text_input.size(1))
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [23]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0038,  0.0513,  0.0233,  0.2895,  0.6321]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**
Highly positive sentiment most probably because of the word bullish.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [24]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [25]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [26]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [27]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.2057,  0.0915,  0.1163,  0.2012,  0.3852]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.